In [ ]:
def get_specific_dataset(api_key, dataset_id):
    url = f"https://apidata.mos.ru/v1/datasets/{dataset_id}"
    params = {'api_key': api_key}

    response = requests.get(url, params=params)
    return response.json()

def search_datasets(api_key, query):
    params = {
        'api_key': api_key,
        '$filter': f"contains(Caption,'{query}')"
    }
    response = requests.get("https://apidata.mos.ru/v1/datasets", params=params)
    return response.json()


In [ ]:
# Ячейка 1 - Ввод и сохранение API ключа
import os

def set_api_key():
    api_key = input("Введите ваш API ключ data.mos.ru: ")
    os.environ['MOS_API_KEY'] = api_key
    return "API ключ успешно сохранен"

# Выполните эту ячейку один раз в начале работы
set_api_key()


Введите ваш API ключ data.mos.ru: f05c436a-520a-4965-9add-95b00876bc56


'API ключ успешно сохранен'

In [ ]:
# Ячейка 2 - Основной код с использованием сохраненного ключа
import requests
import json
import os

def get_moscow_data():
    # Получаем сохраненный API ключ
    api_key = os.environ.get('MOS_API_KEY')

    # Базовый URL
    base_url = "https://apidata.mos.ru/v1/datasets"

    # Параметры запроса
    params = {
        'api_key': api_key,
        '$skip': 1,
        '$top': 1,
        '$inlinecount': 'allpages'
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        print("Статус запроса:", response.status_code)
        print("\nПолученные данные:")
        print(json.dumps(data, indent=2, ensure_ascii=False))

        return data

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

# Теперь можно вызывать функцию многократно без повторного ввода ключа
result = get_moscow_data()


Статус запроса: 200

Полученные данные:
{
  "Items": [],
  "Count": 1230
}


In [ ]:
import requests
import json
import os
from pprint import pprint

def get_datasets_catalog():
    api_key = os.environ.get('MOS_API_KEY')
    base_url = "https://apidata.mos.ru/v1/datasets"

    params = {
        'api_key': api_key,
        '$top': 1000,
        '$inlinecount': 'allpages'  # Добавляем параметр для получения общего количества
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # Получаем Items из ответа
        datasets = data.get('Items', [])
        total_count = data.get('Count', 0)

        print(f"Всего датасетов: {total_count}")
        print(f"Получено датасетов: {len(datasets)}")

        if datasets:
            print("\nПример структуры первого датасета:")
            pprint(datasets[0])

            # Сохраняем в файл для дальнейшего анализа
            with open('moscow_datasets.json', 'w', encoding='utf-8') as f:
                json.dump(datasets, f, ensure_ascii=False, indent=2)

        return datasets

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

# Получаем и сохраняем каталог датасетов
datasets_catalog = get_datasets_catalog()

# Базовый анализ полученных данных
if datasets_catalog:
    # Список уникальных категорий
    categories = set(d.get('CategoryCaption') for d in datasets_catalog)
    print("\nДоступные категории данных:")
    for category in sorted(categories):
        if category:  # Проверяем, что категория не None
            print(f"- {category}")

    # Количество датасетов по категориям
    category_stats = {}
    for d in datasets_catalog:
        cat = d.get('CategoryCaption')
        if cat:  # Учитываем только непустые категории
            category_stats[cat] = category_stats.get(cat, 0) + 1

    print("\nКоличество датасетов в каждой категории:")
    for cat, count in sorted(category_stats.items(), key=lambda x: x[1], reverse=True):
        print(f"{cat}: {count}")



Всего датасетов: 1230
Получено датасетов: 1000

Пример структуры первого датасета:
{'Caption': 'Дома культуры и клубы',
 'CategoryId': 107,
 'ContainsAccEnvData': True,
 'ContainsGeodata': True,
 'DepartmentId': 1008,
 'FullDescription': '<p>Набор данных содержит информацию о домах культуры и '
                    'клубах, подведомственных Департаменту культуры города '
                    'Москвы. Это современные модные пространства, которые '
                    'работают с людьми любого возраста. Их главная идея '
                    '&ndash; позволить жителям столицы развиваться и '
                    'самовыражаться: вокальные ансамбли, ИЗОстудии, '
                    'хореографическая подготовка, спортивные классы и многие '
                    'другие занятия. Выбирай то, что откликнулось тебе, и '
                    'приходи!</p>\n',
 'Id': 493,
 'IdentificationNumber': '7702155262-culturehouse',
 'IsArchive': False,
 'IsForeign': False,
 'IsNew': False,
 'IsSeasonal': False

In [ ]:
# Get list of all datasets
def get_datasets():
    url = "https://apidata.mos.ru/v1/datasets"
    params = {
        'api_key': os.environ.get('MOS_API_KEY'),
        '$top': 1000,
        '$inlinecount': 'allpages'
    }
    return requests.get(url, params=params).json()

# Get specific dataset by ID
def get_dataset(dataset_id):
    url = f"https://apidata.mos.ru/v1/datasets/{dataset_id}"
    params = {'api_key': os.environ.get('MOS_API_KEY')}
    return requests.get(url, params=params).json()

# Get rows from dataset
def get_dataset_rows(dataset_id, top=100):
    url = f"https://apidata.mos.ru/v1/datasets/{dataset_id}/rows"
    params = {
        'api_key': os.environ.get('MOS_API_KEY'),
        '$top': top
    }
    return requests.get(url, params=params).json()

# Get GeoJSON features
def get_geojson_features(dataset_id):
    url = f"https://apidata.mos.ru/v1/datasets/{dataset_id}/features"
    params = {
        'api_key': os.environ.get('MOS_API_KEY'),
        '$top': 1000
    }
    return requests.get(url, params=params).json()

# Get categories
def get_categories():
    url = "https://apidata.mos.ru/v1/categories"
    params = {'api_key': os.environ.get('MOS_API_KEY')}
    return requests.get(url, params=params).json()

# Example usage
datasets = get_datasets()
print(f"Total datasets: {len(datasets)}")

# Get tourist attractions dataset
tourist_data = get_dataset_rows(1903) # Replace with actual dataset ID
print("\nTourist attractions sample:")
print(json.dumps(tourist_data[:2], indent=2, ensure_ascii=False))


Total datasets: 2

Tourist attractions sample:
[
  {
    "global_id": 637376221,
    "Number": 1,
    "Cells": {
      "ID": "00151635",
      "Name": "СМЕТАНА",
      "global_id": 637376221,
      "IsNetObject": "нет",
      "OperatingCompany": "",
      "TypeObject": "кафе",
      "AdmArea": "Северо-Восточный административный округ",
      "District": "Ярославский район",
      "Address": "Российская Федерация, город Москва, внутригородская территория муниципальный округ Ярославский, улица Егора Абакумова, дом 9",
      "PublicPhone": [
        {
          "is_deleted": 0,
          "PublicPhone": "(499) 183-14-10",
          "global_id": 48606
        }
      ],
      "SeatsCount": 44,
      "SocialPrivileges": "нет",
      "Longitude_WGS84": "37.7145650004360390",
      "Latitude_WGS84": "55.8790015313033730",
      "geoData": {
        "coordinates": [
          37.714565,
          55.879001531
        ],
        "type": "Point"
      }
    }
  },
  {
    "global_id": 637376331,


In [ ]:
def find_tourist_datasets():
    """Search for tourism-related datasets"""
    datasets = get_datasets()
    tourist_keywords = ['туризм', 'достопримечательност', 'музей', 'памятник', 'театр']

    tourist_datasets = []
    for dataset in datasets.get('Items', []):
        title = dataset.get('Caption', '').lower()
        if any(keyword in title for keyword in tourist_keywords):
            tourist_datasets.append({
                'id': dataset.get('Id'),
                'title': dataset.get('Caption'),
                'description': dataset.get('Description')
            })

    print("Найдены туристические датасеты:")
    for ds in tourist_datasets:
        print(f"\nID: {ds['id']}")
        print(f"Название: {ds['title']}")
        print(f"Описание: {ds['description']}")

    return tourist_datasets

# Поиск туристических датасетов
tourist_datasets = find_tourist_datasets()

# Получение данных из первого найденного туристического датасета
if tourist_datasets:
    first_dataset = tourist_datasets[0]
    print(f"\nПолучаем данные из датасета: {first_dataset['title']}")
    tourist_data = get_dataset_rows(first_dataset['id'])
    print("\nПример данных:")
    print(json.dumps(tourist_data[:2], indent=2, ensure_ascii=False))


Найдены туристические датасеты:

ID: 495
Название: Кинотеатры
Описание: None

ID: 531
Название: Театры
Описание: None

ID: 1470
Название: Летние кинотеатры
Описание: None

ID: 1803
Название: Магазины «Спорт и туризм»
Описание: None

ID: 2109
Название: График приема граждан: Департамент национальной политики, межрегиональных связей и туризма города Москвы
Описание: None

ID: 2248
Название: Справочник персоналий: Департамент национальной политики, межрегиональных связей и туризма города Москвы
Описание: None

ID: 60789
Название: Wi-Fi в кинотеатрах
Описание: None

Получаем данные из датасета: Кинотеатры

Пример данных:
[
  {
    "global_id": 2639269531,
    "Number": 1,
    "Cells": {
      "Category": "Кинотеатр",
      "CommonName": "«Москино Жуковский»",
      "FullName": "Государственное бюджетное учреждение культуры города Москвы «Московское кино», «Москино Жуковский»",
      "ShortName": "ГБУК города Москвы «Московское кино», Москино, «Москино Жуковский»",
      "OrgInfo": [
      

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium

def create_search_interface():
    # Создаем поле для ввода адреса
    address_input = widgets.Text(
        description='Адрес:',
        placeholder='Введите улицу и дом',
        style={'description_width': 'initial'}
    )

    # Список типов объектов инфраструктуры
    infrastructure_types = [
        'Детский сад',
        'Поликлиника',
        'Больница',
        'Полиция',
        'Метро',
        'Кинотеатр',
        'Концертный зал',
        'Школа',
        'Библиотека',
        'Спортивный центр',
        'Парк',
        'МФЦ'
    ]

    # Создаем выпадающий список
    object_type = widgets.Dropdown(
        options=infrastructure_types,
        description='Тип объекта:',
        style={'description_width': 'initial'}
    )

    # Радиус поиска
    radius_slider = widgets.IntSlider(
        value=1000,
        min=500,
        max=5000,
        step=500,
        description='Радиус поиска (м):',
        style={'description_width': 'initial'}
    )

    # Кнопка поиска
    search_button = widgets.Button(description='Найти')
    output = widgets.Output()

    def on_search_click(b):
        with output:
            clear_output()
            print(f"Поиск {object_type.value} рядом с адресом: {address_input.value}")
            # Здесь будет код для получения координат адреса
            # и поиска объектов через API Москвы

            # Создаем карту
            m = folium.Map(location=[55.7558, 37.6173], zoom_start=11)

            # Здесь будет код для отображения найденных объектов
            display(m)

    search_button.on_click(on_search_click)

    # Собираем интерфейс
    interface = widgets.VBox([
        address_input,
        object_type,
        radius_slider,
        search_button,
        output
    ])

    display(interface)

# Функция для получения данных об объектах
def get_infrastructure_objects(object_type, lat, lon, radius):
    params = {
        'api_key': os.environ.get('MOS_API_KEY'),
        '$filter': f"TypeObject eq '{object_type}'",
        'bbox': f"{lon-0.1},{lat-0.1},{lon+0.1},{lat+0.1}"
    }

    # Здесь будет код для получения данных через API
    return []

# Запускаем интерфейс
create_search_interface()


In [ ]:
# Installations
!pip install ipywidgets folium requests

# Imports
from google.colab import output
output.enable_custom_widget_manager()
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
import requests
import os
import time

def format_moscow_address(address):
    """Форматирование адреса для API Москвы"""
    address_patterns = {
        'улица': 'ул.',
        'проспект': 'пр-т',
        'шоссе': 'ш.',
        'переулок': 'пер.',
        'площадь': 'пл.',
        'бульвар': 'б-р',
        'проезд': 'пр.'
    }
    return f"Москва, {address}"

def search_address(query):
    """Поиск адресов по частичному вводу"""
    url = "https://geocode-maps.yandex.ru/1.x/"
    params = {
        'apikey': 'your_yandex_api_key',  # Замените на ваш API ключ
        'format': 'json',
        'geocode': format_moscow_address(query)
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
        features = data['response']['GeoObjectCollection']['featureMember']
        return [f['GeoObject']['name'] for f in features[:10]]
    except Exception as e:
        print(f"Ошибка поиска: {e}")
        return []

def geocode_address(address):
    """Получение координат по адресу"""
    url = "https://geocode-maps.yandex.ru/1.x/"
    params = {
        'apikey': 'your_yandex_api_key',  # Замените на ваш API ключ
        'format': 'json',
        'geocode': format_moscow_address(address)
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        coords = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
        return {
            'latitude': float(coords[1]),
            'longitude': float(coords[0])
        }
    except Exception:
        return {
            'latitude': 55.7558,  # Координаты центра Москвы
            'longitude': 37.6173
        }

def create_interface():
    # Элементы интерфейса
    address_input = widgets.Combobox(
        placeholder='Введите адрес (например: Красная площадь)',
        description='Адрес:',
        ensure_option=False,
        options=[],
        style={'description_width': 'initial'}
    )

    object_types = [
        'Детский сад', 'Школа', 'Поликлиника', 'Метро', 'Кинотеатр',
        'Больница', 'Полиция', 'МФЦ', 'Библиотека', 'Спортивный центр',
        'Парк', 'Музей', 'Театр', 'Концертный зал', 'Торговый центр',
        'Аптека', 'Почта', 'Банк'
    ]

    object_select = widgets.Dropdown(
        options=object_types,
        description='Тип объекта:',
        style={'description_width': 'initial'}
    )

    radius_slider = widgets.IntSlider(
        value=1000,
        min=500,
        max=5000,
        step=500,
        description='Радиус (м):',
        style={'description_width': 'initial'}
    )

    search_button = widgets.Button(description='Поиск')
    output = widgets.Output()

    def delayed_address_search(query):
        time.sleep(0.5)
        return search_address(query)

    def on_address_change(change):
        if len(change.new) >= 3:
            with output:
                clear_output()
                print("Поиск адресов...")
                suggestions = delayed_address_search(change.new)
                if suggestions:
                    address_input.options = suggestions
                    print(f"Найдено {len(suggestions)} адресов")
                else:
                    print("Уточните адрес")

    def on_button_click(b):
        with output:
            clear_output()
            if not address_input.value:
                print("Введите адрес")
                return

            print("Поиск объектов...")
            coords = geocode_address(address_input.value)
            if coords:
                m = folium.Map(
                    location=[coords['latitude'], coords['longitude']],
                    zoom_start=14
                )
                folium.Marker(
                    [coords['latitude'], coords['longitude']],
                    popup='Ваш адрес',
                    icon=folium.Icon(color='red', icon='info-sign')
                ).add_to(m)

                # Добавляем поиск объектов инфраструктуры в радиусе
                search_nearby_places(m, coords, object_select.value, radius_slider.value)

                display(m)
            else:
                print("Адрес не найден")

    address_input.observe(on_address_change, names='value')
    search_button.on_click(on_button_click)

    interface = widgets.VBox([
        address_input,
        object_select,
        radius_slider,
        search_button,
        output
    ])

    display(interface)

def search_nearby_places(map_obj, coords, place_type, radius):
    """Поиск ближайших объектов инфраструктуры"""
    url = "https://search-maps.yandex.ru/v1/"
    params = {
        'apikey': 'your_yandex_api_key',  # Замените на ваш API ключ для поиска организаций
        'text': place_type,
        'lang': 'ru_RU',
        'll': f"{coords['longitude']},{coords['latitude']}",
        'spn': f"{radius/111000},{radius/111000}",
        'type': 'biz'
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        for obj in data.get('features', []):
            coords = obj['geometry']['coordinates']
            name = obj['properties']['name']
            folium.Marker(
                [coords[1], coords[0]],
                popup=name,
                icon=folium.Icon(color='blue')
            ).add_to(map_obj)
    except Exception as e:
        print(f"Ошибка поиска объектов: {e}")

# Запуск интерфейса
create_interface()




In [ ]:
def get_cultural_places():
    """Получение данных о театрах и домах культуры"""
    url = "https://apidata.mos.ru/v1/datasets/495/rows"
    params = {
        'api_key': os.environ.get('MOS_API_KEY'),
        '$top': 1000
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        # Форматируем данные для отображения
        cultural_places = []
        for place in data:
            cultural_places.append({
                'name': place['Cells']['Name'],
                'address': place['Cells']['Address'],
                'type': place['Cells']['TypeObject'],
                'latitude': float(place['Cells']['Latitude_WGS84']),
                'longitude': float(place['Cells']['Longitude_WGS84']),
                'phone': place['Cells'].get('PublicPhone', [{}])[0].get('PublicPhone', 'Нет телефона')
            })
        return cultural_places
    except Exception as e:
        print(f"Ошибка получения данных: {e}")
        return []

def show_cultural_places_on_map():
    """Отображение театров и домов культуры на карте"""
    places = get_cultural_places()

    # Создаем карту с центром в Москве
    m = folium.Map(location=[55.7558, 37.6173], zoom_start=11)

    # Добавляем маркеры
    for place in places:
        popup_text = f"""
            <b>{place['name']}</b><br>
            Тип: {place['type']}<br>
            Адрес: {place['address']}<br>
            Телефон: {place['phone']}
        """

        folium.Marker(
            [place['latitude'], place['longitude']],
            popup=popup_text,
            icon=folium.Icon(color='purple', icon='info-sign')
        ).add_to(m)

    display(m)
    print(f"Найдено культурных объектов: {len(places)}")

# Запускаем отображение
show_cultural_places_on_map()


Ошибка получения данных: 'Name'


Найдено культурных объектов: 0


In [ ]:
# Installations
!pip install ipywidgets folium requests pyttsx3
!apt-get update && apt-get install -y espeak


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
# Installations
!pip install ipywidgets folium requests pyttsx3
!apt-get update && apt-get install -y espeak

# Imports
from google.colab import output
output.enable_custom_widget_manager()
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import folium
import requests
import os
import time
import pyttsx3
import subprocess

def get_cultural_institutions():
    """Получение данных о культурных учреждениях"""
    datasets = {
        'theaters': {
            'id': '495',
            'name_field': 'CommonName',
            'address_field': 'Address',
            'phone_field': 'PublicPhone',
            'schedule_field': 'WorkingHours'
        },
        'culture_houses': {
            'id': '531',
            'name_field': 'Name',
            'address_field': 'Address',
            'phone_field': 'PublicPhone',
            'schedule_field': 'WorkingHours'
        },
        'museums': {
            'id': '499',
            'name_field': 'Name',
            'address_field': 'Address',
            'phone_field': 'PublicPhone',
            'schedule_field': 'WorkingHours'
        },
        'libraries': {
            'id': '502',
            'name_field': 'Name',
            'address_field': 'Address',
            'phone_field': 'PublicPhone',
            'schedule_field': 'WorkingHours'
        }
    }

    all_places = []

    for institution_type, dataset_info in datasets.items():
        meta_url = f"https://apidata.mos.ru/v1/datasets/{dataset_info['id']}"
        data_url = f"https://apidata.mos.ru/v1/datasets/{dataset_info['id']}/rows"

        headers = {
            'Authorization': 'Bearer YOUR_API_KEY_HERE'
        }

        try:
            meta_response = requests.get(meta_url, headers=headers)
            if meta_response.status_code == 200:
                print(f"Получены метаданные для {institution_type}")

                data_response = requests.get(data_url, headers=headers)
                if data_response.status_code == 200:
                    data = data_response.json()
                    print(f"Успешно загружены данные для {institution_type}: {len(data)} записей")

                    for place in data:
                        cells = place.get('Cells', {})
                        coordinates = cells.get('geoData', {}).get('coordinates', [0, 0])

                        all_places.append({
                            'name': cells.get(dataset_info['name_field'], ''),
                            'type': institution_type,
                            'address': cells.get(dataset_info['address_field'], ''),
                            'phone': cells.get(dataset_info['phone_field'], ''),
                            'schedule': cells.get(dataset_info['schedule_field'], ''),
                            'longitude': coordinates[0] if isinstance(coordinates, list) and len(coordinates) > 0 else 0,
                            'latitude': coordinates[1] if isinstance(coordinates, list) and len(coordinates) > 1 else 0
                        })
                else:
                    print(f"Ошибка получения данных для {institution_type}: {data_response.status_code}")
            else:
                print(f"Ошибка получения метаданных для {institution_type}: {meta_response.status_code}")

        except Exception as e:
            print(f"Ошибка при обработке {institution_type}: {e}")

    return all_places

def speak_text(text):
    """Альтернативный метод озвучивания через espeak"""
    subprocess.run(['espeak', '-v', 'ru', text])

def create_enhanced_interface():
    """Создание расширенного интерфейса с голосовым выводом"""
    institution_types = [
        'Все учреждения',
        'Театры',
        'Дома культуры',
        'Музеи',
        'Библиотеки'
    ]

    type_select = widgets.Dropdown(
        options=institution_types,
        description='Тип учреждения:',
        style={'description_width': 'initial'}
    )

    radius_slider = widgets.IntSlider(
        value=2000,
        min=500,
        max=5000,
        step=500,
        description='Радиус поиска (м):',
        style={'description_width': 'initial'}
    )

    voice_output = widgets.Checkbox(
        value=False,
        description='Голосовой вывод',
        style={'description_width': 'initial'}
    )

    search_button = widgets.Button(description='Найти')
    output = widgets.Output()

    def on_button_click(b):
        with output:
            clear_output()
            print("Поиск учреждений...")

            places = get_cultural_institutions()

            if type_select.value != 'Все учреждения':
                places = [p for p in places if p['type'] == type_select.value.lower()]

            m = folium.Map(location=[55.7558, 37.6173], zoom_start=11)

            for place in places:
                popup_text = f"""
                    <b>{place['name']}</b><br>
                    Тип: {place['type']}<br>
                    Адрес: {place['address']}<br>
                    Телефон: {place['phone']}<br>
                    Расписание: {place['schedule']}
                """

                folium.Marker(
                    [place['latitude'], place['longitude']],
                    popup=popup_text,
                    icon=folium.Icon(color='purple')
                ).add_to(m)

            display(m)

            result_text = f"Найдено {len(places)} учреждений культуры"
            print(result_text)
            if voice_output.value:
                speak_text(result_text)

            for place in places[:5]:
                details = f"\n{place['name']}\nАдрес: {place['address']}\nТелефон: {place['phone']}"
                print(details)
                if voice_output.value:
                    speak_text(details)

    search_button.on_click(on_button_click)

    interface = widgets.VBox([
        type_select,
        radius_slider,
        voice_output,
        search_button,
        output
    ])

    display(interface)

# Запуск интерфейса
create_enhanced_interface()




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
def get_dataset_registry():
    """Получение реестра датасетов"""
    url = "https://apidata.mos.ru/v1/datasets"
    headers = {
        'Authorization': 'Bearer YOUR_API_KEY_HERE'
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            datasets = response.json()
            print(f"Загружено {len(datasets)} датасетов")
            return datasets
        else:
            print(f"Код ответа: {response.status_code}")
            return []
    except Exception as e:
        print(f"Ошибка при загрузке реестра: {e}")
        return []

def display_dataset_info(datasets):
    """Отображение информации о датасетах"""
    for dataset in datasets:
        print(f"\nID: {dataset['Id']}")
        print(f"Название: {dataset['Title']}")
        print(f"Описание: {dataset['Description']}")
        print(f"Количество записей: {dataset['ItemsCount']}")
        print("-" * 50)

def search_datasets(keyword):
    """Поиск датасетов по ключевому слову"""
    datasets = get_dataset_registry()
    found = [d for d in datasets if keyword.lower() in d['Title'].lower() or keyword.lower() in d['Description'].lower()]
    print(f"Найдено {len(found)} датасетов по запросу '{keyword}'")
    display_dataset_info(found)

# Пример использования
search_datasets("культура")


Код ответа: 400
Найдено 0 датасетов по запросу 'культура'


In [ ]:
import pandas as pd
df = pd.read_csv('/content/register.csv')


In [ ]:
df.head()

,property,title,value,format
0,standardversion,Версия методических рекомендаций,https://data.gov.ru/metodicheskie-rekomendacii...,NaN
1,7704786030-metro,Входы и выходы вестибюлей станций Московского ...,http://data.mos.ru/odata/export/meta?id=624&fo...,csv
2,7704786030-busstops,Остановочные пункты,http://data.mos.ru/odata/export/meta?id=752&fo...,csv
3,7704786030-fares,Тарифы на проезд в городском пассажирском тран...,http://data.mos.ru/odata/export/meta?id=658&fo...,csv
4,7704786030-bikeparkings,Велосипедные парковки,http://data.mos.ru/odata/export/meta?id=916&fo...,csv


In [ ]:
df.head(48)

,property,title,value,format
0,standardversion,Версия методических рекомендаций,https://data.gov.ru/metodicheskie-rekomendacii...,NaN
1,7704786030-metro,Входы и выходы вестибюлей станций Московского ...,http://data.mos.ru/odata/export/meta?id=624&fo...,csv
2,7704786030-busstops,Остановочные пункты,http://data.mos.ru/odata/export/meta?id=752&fo...,csv
3,7704786030-fares,Тарифы на проезд в городском пассажирском тран...,http://data.mos.ru/odata/export/meta?id=658&fo...,csv
4,7704786030-bikeparkings,Велосипедные парковки,http://data.mos.ru/odata/export/meta?id=916&fo...,csv
5,7704786030-Routesmobilecomplexes,Маршруты мобильных комплексов фиксации нарушений,http://data.mos.ru/odata/export/meta?id=751&fo...,csv
6,7704786030-Responsibletram,Ответственные балансодержатели межрельсового п...,http://data.mos.ru/odata/export/meta?id=1046&f...,csv
7,7704786030-Interregionalpassengertransport,Смежные маршруты пассажирских перевозок,http://data.mos.ru/odata/export/meta?id=758&fo...,csv
8,7704786030-TaxiParking,Парковки такси,http://data.mos.ru/odata/export/meta?id=621&fo...,csv
9,7704786030-Registerofspeedbumps,Реестр искусственных дорожных неровностей,http://data.mos.ru/odata/export/meta?id=674&fo...,csv


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   property  1231 non-null   object
 1   title     1231 non-null   object
 2   value     1000 non-null   object
 3   format    1230 non-null   object
dtypes: object(4)
memory usage: 38.6+ KB


In [ ]:
# First, let's look at the raw data
import requests

url = 'https://data.mos.ru/opendata/7704786030-fares/data/download?versionNumber=1&releaseNumber=42'
response = requests.get(url)
print("First few lines of raw data:")
print(response.text[:500])

# Now let's try loading with different parameters
df_fares = pd.read_csv(url, encoding='utf-8', sep=',', on_bad_lines='skip')

print("\nDataFrame structure:")
print(df_fares.info())

print("\nFirst few rows:")
print(df_fares.head())


First few lines of raw data:
<!DOCTYPE html>
<html class="notranslate" translate="no" data-critters-container>
<head>
  <meta charset="UTF-8">
  <link rel="shortcut icon" type="image/x-icon" href="assets/svg/logo.svg">
  <meta name="google" content="notranslate">
  <meta name="google-site-verification" content="nyN3q18qWqaEqyyM32j8Pg-IJiGcYg9B9RWEd1OZ0mw">
  <meta name="yandex-verification" content="a63f7d345486875f">
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no

DataFrame structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   <!DOCTYPE html>  27 non-null     object
dtypes: object(1)
memory usage: 344.0+ bytes
None

First few rows:
                                     <!DOCTYPE html>
0  <html class="notranslate" translate="no" data-...
1                                     

In [ ]:
import requests
import pandas as pd
import json

# Using the proper API endpoint
api_url = "https://apidata.mos.ru/v1/datasets/658/rows"
headers = {
    'Authorization': 'Bearer YOUR_API_KEY_HERE'
}

# Get data from API
response = requests.get(api_url, headers=headers)

# Print response status and content type
print("Response status:", response.status_code)
print("Content type:", response.headers.get('content-type'))

# Let's see what we're getting
print("\nFirst part of response:")
print(response.text[:200])

# Parse the response properly
data = response.json()
print("\nData structure:")
print(type(data))
print(data[:1])  # Show first item structure

# Now we can create the correct DataFrame structure


Response status: 400
Content type: application/json

First part of response:
{"errorText":"ApiKey is empty"}

Data structure:
<class 'dict'>


TypeError: unhashable type: 'slice'

Ботаническая коллекция парка «Зарядье»

In [ ]:

# Read CSV with explicit separator
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';')

# Display the first few rows
print("First rows of the dataset:")
print(df_plants.head())

# Display basic information
print("\nDataset information:")
print(df_plants.info())



First rows of the dataset:
    ID                Name           LatinName  \
0  Код            Название  Латинское название   
1    1    Ель обыкновенная         Picea abies   
2    2  Сосна обыкновенная    Pinus sylvestris   
3    3      Дуб черешчатый       Querqus robur   
4    4   Липа мелколистная       Tilia cordata   

                                  Photo   LandscapingZone ProsperityPeriod  \
0                            Фотография  Ландшафтная зона  Период цветения   
1  ba4bd41f-4301-421c-892b-f9b74da0b8de       Хвойный лес            Май\n   
2  102b0915-249e-4c3b-8912-9e484a56db27       Хвойный лес     Май - июнь\n   
3  b0fd9656-54e1-4169-8255-db196c3b44c6     Смешанный лес     Май - июнь\n   
4  eadb690c-3c4c-4878-9b93-1cc032938807     Смешанный лес           Июль\n   

                                         Description         LocationPlace  \
0                                           Описание  Расположение в парке   
1  Ботаники выделяют 40 видов елей. Ель обыкнов

In [ ]:
# First, install espeak
!apt-get update && apt-get install -y espeak

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess

# Load the dataset with correct separator
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';')

# Remove the header row that contains Russian column names
df_plants = df_plants[df_plants['ID'] != 'Код']

def create_plants_interface():
    # Create dropdown with plant names
    plant_select = widgets.Dropdown(
        options=df_plants['Name'].tolist(),
        description='Растение:',
        style={'description_width': 'initial'},
        layout={'width': 'auto'}
    )

    # Create speak button
    speak_button = widgets.Button(
        description='Озвучить',
        button_style='success',
        icon='volume-up'
    )

    output = widgets.Output()

    def speak_text(text):
        """Voice output using espeak with enhanced parameters"""
        try:
            subprocess.run(['espeak', '-v', 'ru', '-s', '150', '-p', '50', text], check=True)
            print("Озвучивание выполнено успешно")
        except subprocess.CalledProcessError as e:
            print(f"Ошибка озвучивания: {e}")

    def on_button_click(b):
        with output:
            clear_output()
            selected_plant = df_plants[df_plants['Name'] == plant_select.value].iloc[0]

            # Simplified text for better voice output
            plant_info = f"Растение {selected_plant['Name']}. {selected_plant['Description']}"

            print("Полная информация:")
            print(f"""
            Название: {selected_plant['Name']}
            Латинское название: {selected_plant['LatinName']}
            Ландшафтная зона: {selected_plant['LandscapingZone']}
            Период цветения: {selected_plant['ProsperityPeriod']}
            Расположение: {selected_plant['LocationPlace']}
            Описание: {selected_plant['Description']}
            """)

            speak_text(plant_info)

    speak_button.on_click(on_button_click)

    # Create interface layout
    interface = widgets.VBox([
        widgets.HTML("<h2>🌳 Растения парка Зарядье</h2>"),
        plant_select,
        speak_button,
        output
    ], layout=widgets.Layout(
        width='80%',
        padding='20px'
    ))

    display(interface)

# Launch interface
create_plants_interface()





Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (186 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [ ]:
# Install required packages
!pip install datasets transformers torch gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import gradio as gr

# Load the dataset
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';', encoding='utf-8')
df_plants = df_plants[df_plants['ID'] != 'Код'].reset_index(drop=True)

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')
model = AutoModelForQuestionAnswering.from_pretrained('sberbank-ai/ruRoberta-large')

def get_plant_info(question):
    for name in df_plants['Name'].unique():
        if name.lower() in question.lower():
            plant = df_plants[df_plants['Name'] == name].iloc[0]

            if 'где' in question.lower():
                return f"{plant['LocationPlace']}"
            elif 'когда' in question.lower() or 'цвет' in question.lower():
                return f"{plant['ProsperityPeriod']}"
            elif 'латинск' in question.lower():
                return f"{plant['LatinName']}"
            else:
                return f"{plant['Description']}"

    return "Растение не найдено в базе данных парка Зарядье"

# Create Gradio interface
iface = gr.Interface(
    fn=get_plant_info,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Введите вопрос (например: Что такое Ель обыкновенная?)"
    ),
    outputs="text",
    title="🌳 Справочник растений парка Зарядье",
    description="Задайте вопрос о растении",
    examples=[
        ["Что такое Ель обыкновенная?"],
        ["Где растет Дуб черешчатый?"],
        ["Когда цветет Липа мелколистная?"],
        ["Латинское название Сосны обыкновенной?"]
    ]
)

iface.launch(share=True)





config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6716b22d9f8c96a33d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Install required packages
!pip install fuzzywuzzy python-Levenshtein gradio transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import gradio as gr
from fuzzywuzzy import fuzz

# Install fuzzy matching
!pip install fuzzywuzzy python-Levenshtein

# Load the dataset
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';', encoding='utf-8')
df_plants = df_plants[df_plants['ID'] != 'Код'].reset_index(drop=True)

def get_plant_info(question):
    # Handle general queries about plant types
    if 'хвойные' in question.lower():
        conifer_plants = df_plants[df_plants['LandscapingZone'].str.contains('Хвойный', na=False)]
        return f"Хвойные растения в парке:\n" + "\n".join(conifer_plants['Name'].tolist())

    if 'все растения' in question.lower() or 'список растений' in question.lower():
        return "Растения в парке:\n" + "\n".join(df_plants['Name'].tolist())

    # Fuzzy matching for plant names
    max_ratio = 0
    matched_name = None
    for name in df_plants['Name'].unique():
        ratio = fuzz.partial_ratio(name.lower(), question.lower())
        if ratio > max_ratio and ratio > 70:  # 70% similarity threshold
            max_ratio = ratio
            matched_name = name

    if matched_name:
        plant = df_plants[df_plants['Name'] == matched_name].iloc[0]

        if any(word in question.lower() for word in ['где', 'расположен', 'растет']):
            return f"{matched_name} расположен в {plant['LocationPlace']}"
        elif any(word in question.lower() for word in ['когда', 'цветет', 'цветение']):
            return f"{matched_name} цветет в период: {plant['ProsperityPeriod']}"
        elif 'латинск' in question.lower():
            return f"Латинское название {matched_name}: {plant['LatinName']}"
        else:
            return f"{plant['Description']}"

    # Handle category-based queries
    if 'смешанный лес' in question.lower():
        mixed_forest = df_plants[df_plants['LandscapingZone'].str.contains('Смешанный', na=False)]
        return f"Растения смешанного леса:\n" + "\n".join(mixed_forest['Name'].tolist())

    return "Уточните вопрос. Вы можете спросить о конкретном растении или группе растений (хвойные, смешанный лес и т.д.)"

# Create enhanced Gradio interface
iface = gr.Interface(
    fn=get_plant_info,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Задайте вопрос о растениях парка"
    ),
    outputs="text",
    title="🌳 Справочник растений парка Зарядье",
    description="Задавайте вопросы о растениях или группах растений",
    examples=[
        ["Какие хвойные растения есть в парке?"],
        ["Где растет Ель?"],
        ["Покажи все растения смешанного леса"],
        ["Когда цветет Липа?"],
        ["Расскажи про Дуб"],
        ["Список всех растений"]
    ]
)

iface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7b66ef96dce47d804a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gTTS

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import gradio as gr
from fuzzywuzzy import fuzz
from gtts import gTTS
import os
import IPython.display as ipd

# Install required packages
!pip install fuzzywuzzy python-Levenshtein gtts

# Load the dataset
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';', encoding='utf-8')
df_plants = df_plants[df_plants['ID'] != 'Код'].reset_index(drop=True)

def text_to_speech(text):
    """Convert text to speech and return audio"""
    tts = gTTS(text=text, lang='ru')
    tts.save('response.mp3')
    return 'response.mp3'

def get_plant_info(question, speak=True):
    # Handle general queries about plant types
    if 'хвойные' in question.lower():
        conifer_plants = df_plants[df_plants['LandscapingZone'].str.contains('Хвойный', na=False)]
        response = f"Хвойные растения в парке:\n" + "\n".join(conifer_plants['Name'].tolist())
        return response, text_to_speech(response) if speak else None

    if 'все растения' in question.lower() or 'список растений' in question.lower():
        response = "Растения в парке:\n" + "\n".join(df_plants['Name'].tolist())
        return response, text_to_speech(response) if speak else None

    # Fuzzy matching for plant names
    max_ratio = 0
    matched_name = None
    for name in df_plants['Name'].unique():
        ratio = fuzz.partial_ratio(name.lower(), question.lower())
        if ratio > max_ratio and ratio > 70:
            max_ratio = ratio
            matched_name = name

    if matched_name:
        plant = df_plants[df_plants['Name'] == matched_name].iloc[0]

        if any(word in question.lower() for word in ['где', 'расположен', 'растет']):
            response = f"{matched_name} расположен в {plant['LocationPlace']}"
        elif any(word in question.lower() for word in ['когда', 'цветет', 'цветение']):
            response = f"{matched_name} цветет в период: {plant['ProsperityPeriod']}"
        elif 'латинск' in question.lower():
            response = f"Латинское название {matched_name}: {plant['LatinName']}"
        else:
            response = f"{plant['Description']}"

        return response, text_to_speech(response) if speak else None

    if 'смешанный лес' in question.lower():
        mixed_forest = df_plants[df_plants['LandscapingZone'].str.contains('Смешанный', na=False)]
        response = f"Растения смешанного леса:\n" + "\n".join(mixed_forest['Name'].tolist())
        return response, text_to_speech(response) if speak else None

    response = "Уточните вопрос. Вы можете спросить о конкретном растении или группе растений"
    return response, text_to_speech(response) if speak else None

# Create enhanced Gradio interface
iface = gr.Interface(
    fn=get_plant_info,
    inputs=[
        gr.Textbox(lines=2, placeholder="Задайте вопрос о растениях парка"),
        gr.Checkbox(label="Озвучить ответ", value=True)
    ],
    outputs=[
        gr.Textbox(label="Текстовый ответ"),
        gr.Audio(label="Аудио ответ")
    ],
    title="🌳 Справочник растений парка Зарядье",
    description="Задавайте вопросы о растениях или группах растений",
    examples=[
        ["Какие хвойные растения есть в парке?", True],
        ["Где растет Ель?", True],
        ["Покажи все растения смешанного леса", True],
        ["Когда цветет Липа?", True],
        ["Расскажи про Дуб", True],
        ["Список всех растений", True]
    ]
)

iface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06ab040cf308aa5422.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
import pandas as pd

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';', encoding='utf-8')
df_plants.head()


,ID,Name,LatinName,Photo,LandscapingZone,ProsperityPeriod,Description,LocationPlace,ViewForm,global_id,Unnamed: 10
0,Код,Название,Латинское название,Фотография,Ландшафтная зона,Период цветения,Описание,Расположение в парке,Форма осмотра,global_id,NaN
1,1,Ель обыкновенная,Picea abies,ba4bd41f-4301-421c-892b-f9b74da0b8de,Хвойный лес,Май\n,Ботаники выделяют 40 видов елей. Ель обыкновен...,Хвойный лес,Бесплатная,881409012,NaN
2,2,Сосна обыкновенная,Pinus sylvestris,102b0915-249e-4c3b-8912-9e484a56db27,Хвойный лес,Май - июнь\n,В средней полосе России исконно растёт лишь од...,Хвойный лес,Бесплатная,881409317,NaN
3,3,Дуб черешчатый,Querqus robur,b0fd9656-54e1-4169-8255-db196c3b44c6,Смешанный лес,Май - июнь\n,Дуб черешчатый считается визитной карточкой ши...,Смешанный лес,Бесплатная,881409921,NaN
4,4,Липа мелколистная,Tilia cordata,eadb690c-3c4c-4878-9b93-1cc032938807,Смешанный лес,Июль\n,В Европейской части России можно встретить чет...,Смешанный лес,Бесплатная,881410524,NaN


In [15]:
# Extract Latin names into a list
latin_names = df_plants['LatinName'].tolist()

print(latin_names)

['Латинское название', 'Picea abies', 'Pinus sylvestris', 'Querqus robur', 'Tilia cordata', 'Acer platanoides', 'Malus baccata', 'Acer saccharinum', 'Sorbus aucuparia', 'Padus avium', 'Betula pendula', 'Betula pubescens', 'Salix alba', 'Salix alba x blanda', 'Salix babylonica «Sverdlovskaja izvilistaja 2»', 'Juniperus communis Hornibrooki', 'Juniperus sabina Mas', 'Pinus pumila', 'Betula nana', 'Salix lanata', 'Salix glauca callicarpaea', 'Coryllus avellana', 'Euonimus verrucosus', 'Sambucus racemosa', 'Rosa canina', 'Spiraea chamaedryfolia', 'Salix ledebouriana x S. purpurea hybrida Sukaszewii', 'Salix purpurea', 'Viburnum opulus', 'Berberis vulgaris', 'Amygdalis nana', 'Cerassus fruticjsa', 'Lonicera tatarica', 'Acer tataricum', 'Syringa vulgaris', 'Rosa rugosa', 'Spiraea bumalda', 'Spiraea vanhouttei', 'Potentilla fruticosa', 'Caltha palustris', 'Carex capillaris', 'Carex capitata', 'Carex caespitosa', 'Cerastium alpinum', 'Dryas octopetala', 'Rubus arcticus', 'Polygonum viviparum',

In [13]:
pip install bing-image-downloader

In [16]:
from bing_image_downloader import downloader
import os
import zipfile

# Download images for each Latin name
for latin_name in latin_names:
    downloader.download(
        latin_name,
        limit=3,  # Number of images per plant
        output_dir='plant_images',
        adult_filter_off=True,
        force_replace=False,
        timeout=60
    )

# Create zip archive
with zipfile.ZipFile('plant_images.zip', 'w') as zipf:
    for latin_name in latin_names:
        image_dir = f'plant_images/{latin_name}'
        if os.path.exists(image_dir):
            for image in os.listdir(image_dir):
                image_path = os.path.join(image_dir, image)
                zipf.write(image_path, f'{latin_name}/{image}')


[%] Downloading Images to /content/plant_images/Латинское название


[!!]Indexing page: 1

[%] Indexed 3 Images on Page 1.


[%] Downloading Image #1 from https://i.pinimg.com/originals/d1/0d/48/d10d48f0eab9086a79bed9f3c301baa4.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://irecommend.ru/sites/default/files/imagecache/copyright1/user-images/1155602/Gq3NoOtTPRyJfLD65tSXw.jpg
[Error]Invalid image, not saving https://irecommend.ru/sites/default/files/imagecache/copyright1/user-images/1155602/Gq3NoOtTPRyJfLD65tSXw.jpg

[!] Issue getting: https://irecommend.ru/sites/default/files/imagecache/copyright1/user-images/1155602/Gq3NoOtTPRyJfLD65tSXw.jpg
[!] Error:: Invalid image, not saving https://irecommend.ru/sites/default/files/imagecache/copyright1/user-images/1155602/Gq3NoOtTPRyJfLD65tSXw.jpg

[%] Downloading Image #2 from https://2.bp.blogspot.com/-gKNdSaxUs4M/WWeNMRkBToI/AAAAAAAAJVA/S0Xup4ymR8kLVxijgsDmCRgxMhZZBxkbgCLcBGAs/s1600/1.jpg
[%] File Downloaded !



[!!]Indexing

In [ ]:
pip install pandas torch transformers gradio fuzzywuzzy python-Levenshtein gtts ipython


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import gradio as gr
from fuzzywuzzy import fuzz
from gtts import gTTS
import os
import glob
import IPython.display as ipd

# Install required packages
!pip install pandas torch transformers gradio fuzzywuzzy python-Levenshtein gtts ipython

# Load the dataset
df_plants = pd.read_csv('/content/data-60861-2024-08-06.csv', sep=';', encoding='utf-8')
df_plants = df_plants[df_plants['ID'] != 'Код'].reset_index(drop=True)

def text_to_speech(text):
    """Convert text to speech and return audio"""
    tts = gTTS(text=text, lang='ru')
    tts.save('response.mp3')
    return 'response.mp3'

def get_plant_images(latin_name):
    """Get image paths for a given plant's Latin name"""
    image_dir = f'/content/plant_images/{latin_name}'
    if os.path.exists(image_dir):
        return sorted(glob.glob(f'{image_dir}/Image_*.jpg'))
    return []

def get_plant_info(question, speak=True):
    # Handle general queries about plant types
    if 'хвойные' in question.lower():
        conifer_plants = df_plants[df_plants['LandscapingZone'].str.contains('Хвойный', na=False)]
        response = f"Хвойные растения в парке:\n" + "\n".join(conifer_plants['Name'].tolist())
        return response, text_to_speech(response) if speak else None, []

    if 'все растения' in question.lower() or 'список растений' in question.lower():
        response = "Растения в парке:\n" + "\n".join(df_plants['Name'].tolist())
        return response, text_to_speech(response) if speak else None, []

    # Fuzzy matching for plant names
    max_ratio = 0
    matched_name = None
    for name in df_plants['Name'].unique():
        ratio = fuzz.partial_ratio(name.lower(), question.lower())
        if ratio > max_ratio and ratio > 70:
            max_ratio = ratio
            matched_name = name

    if matched_name:
        plant = df_plants[df_plants['Name'] == matched_name].iloc[0]
        latin_name = plant['LatinName']
        images = get_plant_images(latin_name)

        if any(word in question.lower() for word in ['где', 'расположен', 'растет']):
            response = f"{matched_name} расположен в {plant['LocationPlace']}"
        elif any(word in question.lower() for word in ['когда', 'цветет', 'цветение']):
            response = f"{matched_name} цветет в период: {plant['ProsperityPeriod']}"
        elif 'латинск' in question.lower():
            response = f"Латинское название {matched_name}: {plant['LatinName']}"
        else:
            response = f"{plant['Description']}"

        return response, text_to_speech(response) if speak else None, images

    if 'смешанный лес' in question.lower():
        mixed_forest = df_plants[df_plants['LandscapingZone'].str.contains('Смешанный', na=False)]
        response = f"Растения смешанного леса:\n" + "\n".join(mixed_forest['Name'].tolist())
        return response, text_to_speech(response) if speak else None, []

    response = "Уточните вопрос. Вы можете спросить о конкретном растении или группе растений"
    return response, text_to_speech(response) if speak else None, []

# Create enhanced Gradio interface
iface = gr.Interface(
    fn=get_plant_info,
    inputs=[
        gr.Textbox(lines=2, placeholder="Задайте вопрос о растениях парка"),
        gr.Checkbox(label="Озвучить ответ", value=True)
    ],
    outputs=[
        gr.Textbox(label="Текстовый ответ"),
        gr.Audio(label="Аудио ответ"),
        gr.Gallery(label="Изображения растения")
    ],
    title="🌳 Справочник растений парка Зарядье",
    description="Задавайте вопросы о растениях или группах растений. Для каждого растения доступны фотографии.",
    examples=[
        ["Какие хвойные растения есть в парке?", True],
        ["Где растет Ель?", True],
        ["Покажи все растения смешанного леса", True],
        ["Когда цветет Липа?", True],
        ["Расскажи про Дуб", True],
        ["Список всех растений", True]
    ]
)

iface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c5ef6b524ce009a08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
